#Prerequisite

####Load employee.csv into DataFrame

In [0]:
df = spark.read.csv(path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/employee.csv",
                    sep="|",
                    inferSchema=True,
                    header=True,
                    quote="'")
display(df)  

##Select Operations

In [0]:
df.select("id","name").display()

###Recommended way to select a columns from DF is through Column object

In [0]:
from pyspark.sql.functions import col
df.select(col("id").alias("user_id"),col("id"),col("name")).display()

###Filter only records whose designation as developer

In [0]:
df.filter(col("desig")=="Developer").display()

In [0]:
df.filter((col("desig")=="Developer") |(col("desig")=="Team Lead")).display()

df.filter((col("desig")=="Developer") |(col("desig")=="Team Lead")).select("id").display()

In [0]:
df.filter((col("desig")=="Developer") & (col("gen")=="M")).display()

In [0]:
df.filter(col("exp").between(1,10)).display()

In [0]:
df.filter((col("exp").between(1,6)) & (col("gen")=="M")).select(col("name")).display()

###Grouping, Aggregation and sorting

In [0]:
df.groupBy(col("gen")).count().display()

In [0]:
#df.groupBy(col("gen")).count().sort(col("count"),ascendig=False).display()
df.groupBy(col("gen")).count().sort(col("count").desc()).display()

In [0]:
#df.groupBy(col("gen")).count().sort(col("count"), ascending=False).display()
df.groupBy(col("gen")).count().sort(col("count"), ascending=False).display()

 

###min, max are taking string col ,so we can use alias methid directly

In [0]:
from pyspark.sql.functions import min,max,avg,sum
df.groupBy(col("desig")).agg(min("exp").alias("min_exp"),
                             max("exp").alias("max_exp"),
                             avg("exp").alias("avg_exp")).display()

##Derive New Columns

In [0]:
from pyspark.sql.functions import lit
df.withColumn("is_employeed",lit("Yes")).display()

In [0]:
from pyspark.sql.functions import when

df.withColumn(
    "Category",
    when(col("exp") >= 10, "Senior")
    .when(col("exp") >= 5, "Mid Level")
    .when(col("exp") >= 1, "Junior")
    .otherwise("Invalid Experience"),
).display()

#View

In [0]:
df.createOrReplaceTempView("employee_vw")

In [0]:
%sql
SELECT *,
  CASE
    WHEN EXP >=10 THEN 'Senior'
    when exp >=5 THEN 'Mid'
    WHEN exp >=0 THEN 'Junior'
    ELSE 'Invalid Experience'
END AS category
FROm employee_vw